<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

# Python for Finance Course

**Module 6 &mdash; Performance**

[team@tpq.io](mailto:team@tpq.io) | [http://tpq.io](http://tpq.io)

The Python Quants GmbH

## Requirements

Make sure to have installed locally a **comprehensive Python installation** like the [Anaconda](http://continuum.io/downloads) Python distribution.

Alternatively, and more easily, register on the **[Quant Platform](http://pqp.io)** where you can execute this Jupyter Notebook file in the browser. After registration, you find all Jupyter Notebook files of this course in the folder `pffcourse`.

## Necessary Imports

As usual, we need to **import certain modules/packages**.

In [1]:
import math  # mathematical functions
import numpy as np  # array operations
import pandas as pd  # time series management
import numba as nb  # dynamic compiling

## Nested Loops

This module is about selected topics in **Performance Python**. Some people might still believe in the following logic:

* many financial algorithms require (nested) loops when implemented
* since Python is interpreted, it is slow with (nested) loops
* therefore, Python is not well suited for the implementation of financial algorithms

Let us analyze a sandbox case of a nested loop that has nothing to do with finance but that illustrates the issues at hand quite well.

### Pure Python

Consider the following nested loop which only **counts the number of iterations**. To make it a bit more demanding (actually quite a lot more), the number of iterations in the inner loop increases with the current state of the outer loop.

In [2]:
%%time
def count(N):
    s = 0
    for i in xrange(N):
        for j in xrange(N * i):
            # generate intentionally some computational burden
            s += int(math.log(math.sin(math.pi / 2)) + 1)
    return s
print count(250)

7781250
CPU times: user 3.05 s, sys: 47 ms, total: 3.09 s
Wall time: 3 s


Although in principle possible, such problems do not lend themselves very well to be vectorized in order to speed things up. The memory burden easily becomes prohibitive.

### Dynamic Compiling with Numba

Numba is a Python library that allows to **dynamically compile Python code** to machine code (using the [LLVM infrastructure](https://en.wikipedia.org/wiki/LLVM)). The application is straightforward.

In [3]:
count_nb = nb.jit(count)

The result of the compilation is a **directly callable function**. First time execution involves **a bit of overhead**.

In [4]:
%time count_nb(250)

CPU times: user 45 ms, sys: 2 ms, total: 47 ms
Wall time: 393 ms


7781250

From the second call, you get the **complete (time) benefits**.

In [5]:
%time count_nb(250)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 19.1 µs


7781250

### Improved Python 

A part of the huge speed-up comes from the **avoidance of recalculating the complex inner expression** over and over again. Therefore, to be fair let us use this 'trick' with pure Python as well.

In [6]:
%%time
def count(N):
    s = 0
    ex = int(math.log(math.sin(math.pi / 2)))
    for i in xrange(N):
        for j in xrange(N * i):
            # now a simple addition
            s += ex + 1
    return s
print count(250)

7781250
CPU times: user 285 ms, sys: 0 ns, total: 285 ms
Wall time: 284 ms


This reduces the execution time considerably. However, the **compiled version is still multiple orders of magnitude faster** than the improved Python version.

## Recursive Algorithms

**Vectorization** is a popular strategy to speed up code with NumPy and/or pandas. However, not all types of algorithms allow for (easy) vectorization. In this section, we apply dynamic compiling to such a problem.

### The Algorithm

Consider the following **simple, recursive algorithm**. For $i=0$ (first element), we have

$$
r_0 = s_0
$$

For $i>0$ (all others), we have

$$
r_i = r_{i-1} + \frac{1}{2} s_{i}
$$

In Python, we get something like the following code.

In [7]:
def simple(s):
    r = np.empty_like(s)
    for i in xrange(len(s)):
        if i == 0:
            r[i] = s[i]
        else:
            r[i] = r[i - 1] + 0.5 * s[i]
    return r

As a **quick illustration**, consider the following small `ndarray` object.

In [8]:
a = np.arange(9.)
a

array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.])

Application of the function yields:

In [9]:
%time simple(a)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 21.9 µs


array([  0. ,   0.5,   1.5,   3. ,   5. ,   7.5,  10.5,  14. ,  18. ])

Application of this function to a **much larger `ndarray` object** is equally straightforward.

In [10]:
data = np.arange(10000000.)

In [11]:
%time simple(data)

CPU times: user 3.03 s, sys: 78 ms, total: 3.11 s
Wall time: 2.97 s


array([  0.00000000e+00,   5.00000000e-01,   1.50000000e+00, ...,
         2.49999875e+13,   2.49999925e+13,   2.49999975e+13])

### Numba Version

First, we **compile** the function.

In [12]:
simple_nb = nb.jit(simple)

**Quick check** whether it works.

In [13]:
simple_nb(a)

array([  0. ,   0.5,   1.5,   3. ,   5. ,   7.5,  10.5,  14. ,  18. ])

Finally, the application to the **larger object**. The speedup in this particular case is again orders of magnitude large.

In [14]:
%time simple_nb(data)

CPU times: user 24 ms, sys: 6 ms, total: 30 ms
Wall time: 30.5 ms


array([  0.00000000e+00,   5.00000000e-01,   1.50000000e+00, ...,
         2.49999875e+13,   2.49999925e+13,   2.49999975e+13])

## Closing Remarks

This finishes the sixth module of the Python for Finance email course. In this module, you have learned:

* that (nested) loops in Python can be slow
* that you should separate compute intensive calculations from loops whenever possible
* that dynamic compiling with Numba can speed up (nested) loops by several orders of magnitude
* that not all (financial) algorithms (e.g. those with recursion) can be vectorized (easily)
* that dynamic compiling with Numba also might speed-up algorithms that use recursion

Many financial algorithms applied in practice are time critical in the sense that their speed of execution is a major factor. Dynamic compiling is one approach that might &mdash; without much overhead &mdash; yield significant performance improvements for Python code.

## Exercises

In order to master the material of this module, do the following:

* think of different loop structures seen in or similar to financial algorithms
* implement them in pure Python, in vectorized fashion (if possible) and dynamically compiled
* implement the binomial option pricing model in pure Python, vectorized and dynamically compiled
* implement a Monte Carlo simulation for a geometric Brownian motion in pure Python, vectorized and dynamically compiled
* compare the performance of the different implementations

## References

You find background information for the topics covered in this module in the following book:

* Hilpisch, Yves (2014): _Python for Finance_. O'Reilly, ch. 8.

<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

<a href="http://tpq.io" target="_blank">http://tpq.io</a> | <a href="http://twitter.com/dyjh" target="_blank">@dyjh</a> | <a href="mailto:team@tpq.io">team@tpq.io</a>

**Quant Platform** |
<a href="http://quant-platform.com">http://quant-platform.com</a>

**Python for Finance** |
<a href="http://python-for-finance.com" target="_blank">Python for Finance @ O'Reilly</a>

**Derivatives Analytics with Python** |
<a href="http://derivatives-analytics-with-python.com" target="_blank">Derivatives Analytics @ Wiley Finance</a>

**Listed Volatility and Variance Derivatives** |
<a href="http://lvvd.tpq.io" target="_blank">Listed VV Derivatives @ Wiley Finance</a>